In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from argparse import Namespace
from collections import defaultdict
import copy
import difflib
import gzip
import itertools
import os
import pickle
import sys
import typing

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

from IPython.display import display, Markdown, HTML
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline
import tatsu
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm.notebook as tqdm


sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('../reward-machine'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.ast_counter_sampler import *
from src.evolutionary_sampler import *
from src.ast_utils import cached_load_and_parse_games_from_file, load_games_from_file, _extract_game_id
from src import ast_printer
from latest_model_paths import MAP_ELIETS_MODELS, LATEST_FITNESS_FUNCTION_DATE_ID

In [ ]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]
regrown_game_1024_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples-1024.pddl.gz'))
print(len(real_game_texts), len(regrown_game_1024_texts), len(regrown_game_1024_texts) / 98)


In [ ]:
def extract_game_index(game_name: str):
    first_dash = game_name.find('-')
    second_dash = game_name.find('-', first_dash + 1)
    index = game_name[first_dash + 1:second_dash] if second_dash != -1 else game_name[first_dash + 1:]
    return int(index)


def extract_negative_index(game_name: str):
    first_dash = game_name.find('-')
    second_dash = game_name.find('-', first_dash + 1)
    if second_dash == -1:
        return -1
    
    third_dash = game_name.find('-', second_dash + 1)
    index = game_name[second_dash + 1:third_dash]
    return int(index)


fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')

# fitness_df = fitness_df.assign(real=fitness_df.real.astype('int'), game_index=fitness_df.game_name.apply(extract_game_index), 
#                                negative_index= fitness_df.game_name.apply(extract_negative_index), fake=~fitness_df.real.astype('int'))
# fitness_df = fitness_df.sort_values(by=['fake', 'game_index', 'negative_index'], ignore_index=True).reset_index(drop=True)
# fitness_df.drop(columns=['Index', 'fake', 'game_index', 'negative_index'], inplace=True)
print(fitness_df.src_file.unique())
fitness_df.head()

In [ ]:
# Change here if you want to use a different model
model_date_id = LATEST_FITNESS_FUNCTION_DATE_ID
data_df = fitness_df
cv_energy_model, feature_columns = utils.load_model_and_feature_columns(model_date_id)
print(len(feature_columns))


## Real Games

In [ ]:
def _display_game(title: str, game_text: str):
    display(Markdown(f'### {title}:'))
    display(Markdown(f'```pddl\n{game_text}\n```'))


def dispaly_real_game(index: int):
    title = f'Game #{index} ({game_asts[index][1].game_name})'
    game_text = real_game_texts[index]
    _display_game(title, game_text)


dispaly_real_game(0)

## MAP-Elites Games

In [ ]:
# keys and specs defined in `latest_model_paths.MAP_ELIETS_MODELS`

map_elites_model_key = 'previous_object_and_predicates_with_expected_values_bc'
map_elites_model_spec = MAP_ELIETS_MODELS[map_elites_model_key]
map_elites_model = typing.cast(MAPElitesSampler, map_elites_model_spec.load())


display_overall_features = False   # set to true to show which features drive energy most up/down

# If you want one of the top samples, by descending index
map_elites_model.visualize_top_sample(1, display_overall_features=display_overall_features)

# If you want a random sample
# map_elites_model.visualize_random_sample(display_overall_features=display_overall_features)

# If you want a specific sample by key
# key = (1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1)
# map_elites_model._visualize_sample_by_key(key, display_overall_features=display_overall_features)


## Regrown negatives

In [ ]:
full_tensor = utils.df_to_tensor(data_df, feature_columns)
if 'wrapper' in cv_energy_model.named_steps: cv_energy_model.named_steps['wrapper'].eval()
full_tensor_scores = cv_energy_model.transform(full_tensor).detach()

real_game_scores = full_tensor_scores[:, 0]

print(f'Real game scores: {real_game_scores.mean():.4f} ± {real_game_scores.std():.4f}, min = {real_game_scores.min():.4f}, max = {real_game_scores.max():.4f}')

negatives_scores = full_tensor_scores[:, 1:]
torch.quantile(negatives_scores.ravel(), torch.linspace(0, 1, 11))
print(f'20th percentile negative energy: {torch.quantile(negatives_scores.ravel(), 0.2)}')

In [ ]:
steps = torch.linspace(0, 1, 101)
percentiles = torch.quantile(negatives_scores.ravel(), steps)
print(steps)
print(f'Energy percentiles: {steps}')

In [ ]:
def print_negatives_from_quantile(quantile: int, n_games: int, output_dir: str = './temp_outputs/'):
    min_energy = percentiles[20 * quantile]
    max_energy = percentiles[20 * (quantile + 1)]
    quantile_indices = ((negatives_scores >= min_energy) & (negatives_scores <= max_energy)).nonzero()
    game_indices = torch.randperm(quantile_indices.shape[0])[:n_games]
    lines = [f'; Games from quantile #{quantile} with energies in the range [{min_energy:.4f} ({int(steps[2 * quantile] * 100)}%), {max_energy:.4f} ({int(steps[2 * (quantile + 1)] * 100)}%) ]']
    for idx in game_indices:
        overall_index = 98 * quantile_indices[idx, 0] + quantile_indices[idx, 1]
        lines.append(f'; Game with index {overall_index} and energy {negatives_scores[quantile_indices[idx, 0], quantile_indices[idx, 1], quantile_indices[idx, 2]]:.4f}')
        lines.append(regrown_game_1024_texts[overall_index])
        lines.append('')

    with open(os.path.join(output_dir, f'quantile_{quantile}.txt'), 'w') as f:
        f.write('\n'.join(lines))


# n_games_to_print = 25
# for q in range(5):
#     print_negatives_from_quantile(q, n_games_to_print)
    


In [ ]:
RANDOM_SEED = 33
torch.manual_seed(RANDOM_SEED)

N_GAMES = 20
N_GAMES_PER_QUANTILE = 10
N_QUANTILES = 4

original_game_indices = list(torch.randperm(full_tensor.shape[0])[:N_GAMES].numpy())
regrowths_by_original_game_index_and_quantile = {idx: {} for idx in original_game_indices}

for game_idx in original_game_indices:
    for q in range(N_QUANTILES):
        quantile_step = int(100 / N_QUANTILES)
        min_energy = percentiles[quantile_step * q]
        max_energy = percentiles[quantile_step * (q + 1)]
        game_negatives = negatives_scores[game_idx].squeeze()
        quantile_indices = ((game_negatives >= min_energy) & (game_negatives <= max_energy)).nonzero().squeeze()
        negatives_in_quantile_indices = torch.randperm(quantile_indices.shape[0])[:N_GAMES_PER_QUANTILE]
        regrowths_by_original_game_index_and_quantile[game_idx][q] = list(quantile_indices[negatives_in_quantile_indices].numpy())

        

In [ ]:
import csv
from game_describer import GameDescriber
describer = GameDescriber(grammar_path='../dsl/dsl.ebnf')


In [ ]:
game_idx = 0  # 0 to N_GAMES - 1
quantile = 3  # 0 to N_QUANTILES - 1
negative_idx = 0  # 0 to N_GAMES_PER_QUANTILE - 1

with open("./test.csv", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["SECTION", "STAGE 0", "STAGE 1", "STAGE 2"])
    for game_idx in range(0, 20):
        # for quantile in range(0, 4):
        quantile = game_idx % 4
        print(f"Testing game {game_idx}, quantile {quantile}")
        original_game_index = original_game_indices[game_idx]
        original_game = real_game_texts[original_game_index]
        regrowth_game = regrown_game_1024_texts[original_game_index * 1024 + regrowths_by_original_game_index_and_quantile[original_game_index][quantile][negative_idx]]

        stage_0 = describer.describe_stage_0(regrowth_game)
        stage_1 = describer.describe_stage_1(regrowth_game)

        for section_idx, section in enumerate(["SETUP", "PREFERENCES", "TERMINAL", "SCORING"]):
            writer.writerow([f"[Game {game_idx}] [Quantile {quantile}] {section}", stage_0[section_idx], stage_1[section_idx], ""])

        writer.writerow(["", "", "", ""])

        

In [ ]:
def visualize_game_diff(game_idx: int, quantile: int, negative_idx: int, output_dir: str = './temp_outputs/'):
    original_game_index = original_game_indices[game_idx]
    original_game = real_game_texts[original_game_index]
    regrowth_game = regrown_game_1024_texts[original_game_index * 1024 + regrowths_by_original_game_index_and_quantile[original_game_index][quantile][negative_idx]]

    utils.display_game_diff_html(original_game, regrowth_game)


game_index = 0  # 0 to N_GAMES - 1
quantile = 3  # 0 to N_QUANTILES - 1
negative_index = 0  # 0 to N_GAMES_PER_QUANTILE - 1

visualize_game_diff(game_index, quantile, negative_index)
    